In [ ]:
from pydantic import BaseModel
import torch

In [ ]:
# This part of code will skip all the un-necessary warnings which can occur during the execution of this project.
import warnings
warnings.filterwarnings("ignore", category=Warning)

In [ ]:
# Installation for GPU llama-cpp-python==0.2.69
!CMAKE_ARGS="-DLLAMA_CUDA=on" pip install llama-cpp-python==0.2.69
# For downloading the models from HF Hub
!pip install huggingface_hub

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.5/42.5 MB 9.9 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 4.3 MB/s eta 0:00:00
  Created wheel for llama-cpp-python: filename=llama_cpp_python-0.2.69-cp311-cp311-linux_x86_64.whl size=55715097 sha256=9adb871902ce04cc7db3d975f395b6baaf6b0e413f003c5096505b303c92d534
  Stored in directory: /root/.cache/pip/wheels/e8/1b/ff/b4dba97fbd16e731705b262602ba8f3b672bf4bde54ea0c104
Successfully built llama-cpp-python


In [ ]:
# !pip uninstall llama-cpp-python
# CMAKE_ARGS="-DLLAMA_CUBLAS=on"
# !pip install --force-reinstall llama-cpp-python --no-cache-dir

In [ ]:
import json

import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split

from huggingface_hub import hf_hub_download
from llama_cpp import Llama

from tqdm import tqdm
from collections import Counter

In [ ]:
# Load Mistral 7B model (Placeholder for actual model loading)
def load_model():
    model_name_or_path = "TheBloke/Mistral-7B-Instruct-v0.2-GGUF"
    model_basename = "mistral-7b-instruct-v0.2.Q5_K_M.gguf"
    model_path = hf_hub_download(
              repo_id=model_name_or_path,
              filename=model_basename
          )
    lcpp_llm = Llama(

            model_path=model_path,
            n_threads=2, # CPU cores
            n_batch=512, # Should be between 1 and n_ctx, consider the amount of VRAM in your GPU.
            n_gpu_layers=43, # Change this value based on your model and your GPU VRAM pool.
            n_ctx=4096 # Context window
        )
    return lcpp_llm

model = load_model()

mistral-7b-instruct-v0.2.Q5_K_M.gguf:   0%|          | 0.00/5.13G [00:00<?, ?B/s]

llama_model_loader: loaded meta data with 24 key-value pairs and 291 tensors from /root/.cache/huggingface/hub/models--TheBloke--Mistral-7B-Instruct-v0.2-GGUF/snapshots/3a6fbf4a41a1d52e415a4958cde6856d34b2db93/mistral-7b-instruct-v0.2.Q5_K_M.gguf (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.name str              = mistralai_mistral-7b-instruct-v0.2
llama_model_loader: - kv   2:                       llama.context_length u32              = 32768
llama_model_loader: - kv   3:                     llama.embedding_length u32              = 4096
llama_model_loader: - kv   4:                          llama.block_count u32              = 32
llama_model_loader: - kv   5:                  llama.feed_forward_length u32              = 14336
llama_model_loa

In [ ]:
!pip install flask flask-ngrok
!pip install pyngrok

In [ ]:
from pydantic import BaseModel
from flask import Flask, jsonify, request
from flask_ngrok import run_with_ngrok  # Ensure you have flask-ngrok installed
from pyngrok import ngrok

app = Flask(__name__)
run_with_ngrok(app)  # Enables ngrok

!ngrok authtoken 2flPCqXtK0mJarxJ8qKyWAdyP8P_5jAr786C4PjUnW9QWZ2kZ

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [ ]:
class PredictionRequest(BaseModel):
  message: str
  max_tokens: int = 200

In [ ]:
mistral_prompt_template = """<s>[INST]{prompt}[/INST]"""

answers_template = """
Context:
{context}
===
Using the context above generate {num_answers} distinct answers to the following question:
Question:
{question}.

Arrange your answers in numbered bullet points.
Present only the answers in bullet points.
"""

In [ ]:
def generateModelResponse(user_query, user_query_context):
  answers_prompt = mistral_prompt_template.format(
      prompt=answers_template.format(
          context=user_query_context,
          question=user_query,
          num_answers=3
      )
  )

  response = model(
    prompt=answers_prompt,
    max_tokens=1024,
    temperature=0,
    top_p=0.95,
    repeat_penalty=1.2,
    echo=False # do not return the prompt
  )
  factual_answers = response["choices"][0]["text"].strip()
  return factual_answers

In [ ]:
{"Predictive insights": generateModelResponse("I'm very happy with the product", "You are an AI assistant that performs sentiment analysis.")}

In [ ]:
@app.post("/api/v1/predictiveInsight")
def predictive_insights():
    input = request.get_json()
    system_message = "You are an AI assistant that generates predictive insights."
    response = generateModelResponse(input.get("message", ""), system_message)
    return {"Predictive insights": response}

In [ ]:
@app.post("/api/v1/sentiment")
def sentiment_analysis():
    input = request.get_json()
    system_message = "You are an AI assistant that performs sentiment analysis."
    response = generateModelResponse(input.get("message", ""), system_message)
    return {"User sentiment": response}

In [ ]:
@app.post("/api/v1/personalizeRecommendation/")
def personalized_recommendation():
    input = request.get_json()
    system_message = "You are an AI assistant that provide personalized recommendations."
    response = generateModelResponse(input.get("message", ""), system_message)
    return {"recommendation": response}

In [ ]:
@app.post("/api/v1/assessRisk/")
def risk_assessment():
    input = request.get_json()
    system_message = "You are an AI assistant that performs risk assessments."
    response = generateModelResponse(input.get("message", ""), system_message)
    return {"Risk_assessment": response}

In [ ]:
@app.route("/", methods=["GET"])
def root():
    return {"message": "AI-driven API for predictive insights, personalization, sentiment analysis, and risk assessment"}

In [ ]:
# Open a tunnel on the port Flask is running on (default 5000)
public_url = ngrok.connect(5000)
print(" * Tunnel URL:", public_url)

if __name__ == "__main__":
    app.run()

 * Tunnel URL: NgrokTunnel: "https://86f0-34-125-62-208.ngrok-free.app" -> "http://localhost:5000"
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit


 * Running on http://86f0-34-125-62-208.ngrok-free.app
 * Traffic stats available on http://127.0.0.1:4040


Llama.generate: prefix-match hit

llama_print_timings:        load time =     534.48 ms
llama_print_timings:      sample time =      29.52 ms /    57 runs   (    0.52 ms per token,  1931.16 tokens per second)
llama_print_timings: prompt eval time =     227.00 ms /    33 tokens (    6.88 ms per token,   145.38 tokens per second)
llama_print_timings:        eval time =    1644.44 ms /    56 runs   (   29.37 ms per token,    34.05 tokens per second)
llama_print_timings:       total time =    2036.75 ms /    89 tokens
INFO:werkzeug:127.0.0.1 - - [26/Mar/2025 07:07:52] "POST /api/v1/sentiment HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [26/Mar/2025 07:08:10] "POST /api/v1/personalizeRecommendation HTTP/1.1" 308 -
Llama.generate: prefix-match hit

llama_print_timings:        load time =     534.48 ms
llama_print_timings:      sample time =      63.44 ms /   118 runs   (    0.54 ms per token,  1860.11 tokens per second)
llama_print_timings: prompt eval time =     217.01 ms /    64 tokens (   